In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/netflix-prize-data/combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1],low_memory=True)

In [ ]:
data

In [ ]:
index=data[data['Rating'].isna()]

In [ ]:
index

In [ ]:
movie_id=np.zeros(data.shape[0])

In [ ]:
j=1
for i in index.index:
    movie_id[i:]=j
    j+=1

In [ ]:
data.shape,movie_id.shape

In [ ]:
data['Movie_id']=movie_id

In [ ]:
X=data.dropna()

In [ ]:
X

In [ ]:
plt.pie(X['Rating'].value_counts(),labels=['5','4','3','2','1'])

In [ ]:
X['Movie_id'].value_counts()

In [ ]:
X['Cust_Id'].value_counts()

In [ ]:
X[X['Rating']==1]['Movie_id'].value_counts()

In [ ]:
cust_group=X.groupby(['Cust_Id'])['Rating'].count()

In [ ]:
cust_ids=cust_group[cust_group>100]

In [ ]:
ids=pd.DataFrame(cust_ids)

In [ ]:
ids['Cust_Id']=cust_ids.index

In [ ]:
x_join=X.join(ids,how='inner',on=['Cust_Id'],rsuffix='r_')

In [ ]:
movie_ids=pd.DataFrame(x_join.groupby('Movie_id')['Rating'].count())

In [ ]:
movie_ids['Movie_id']=movie_ids.index

In [ ]:
movie_ids=movie_ids[movie_ids['Rating']>500]

In [ ]:
x_join2=x_join.join(movie_ids,how='inner',on='Movie_id',rsuffix='r_')

In [ ]:
train_data=x_join2[['Cust_Id','Movie_id','Rating']]

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data_matrix = Dataset.load_from_df(train_data[['Cust_Id','Movie_id','Rating']], reader)

In [ ]:
trainset, testset = train_test_split(data_matrix, test_size=.25)

In [ ]:
algo=SVD()

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
accuracy.rmse(predictions)

In [ ]:
accuracy.mae(predictions)